In [5]:
!pip install mediapipe opencv-python pyautogui


In [6]:
!pip install keyboard


### Change 1 Slide to the Right

In [8]:
import pygetwindow as gw
import pyautogui
import time
import keyboard  # Alternative for key presses

def activate_presentation_window():
    """
    Activate the PowerPoint Slide Show window.
    Returns True if the window is found and activated, otherwise False.
    """
    for window in gw.getAllTitles():
        if "Slide Show" in window:  # Match partial title
            target_window = gw.getWindowsWithTitle(window)[0]
            print(f"Activating window: {target_window.title}")
            target_window.activate()
            return True
    print("Slide Show window not found. Ensure it is open and in full-screen mode.")
    return False

def send_key_press(key):
    """
    Send a key press to the active window.
    Tries pyautogui first, and falls back to keyboard if necessary.
    """
    try:
        pyautogui.press(key)  # Simulates key press
        print(f"pyautogui: '{key}' key pressed")
    except Exception as e:
        print(f"pyautogui failed: {e}. Trying keyboard library...")
        try:
            keyboard.send(key)  # Alternative using keyboard library
            print(f"keyboard: '{key}' key pressed")
        except Exception as ex:
            print(f"keyboard library failed: {ex}. Unable to send '{key}' key press.")

# Wait for the user to switch to the presentation window
print("You have 5 seconds to switch to the PowerPoint Slide Show window...")
time.sleep(5)  # Gives you time to switch to the presentation window

# Try to activate the Slide Show window
if activate_presentation_window():
    # Send the "right" arrow key press to move to the next slide
    send_key_press('right')
else:
    print("Failed to activate the presentation window. Exiting.")


You have 5 seconds to switch to the PowerPoint Slide Show window...
Activating window: PowerPoint Slide Show  -  Participant_01
pyautogui: 'right' key pressed


### Change Slides to Right with Right Index / Left with Left Index finger shown on the camera

In [9]:
import cv2
import mediapipe as mp
import pyautogui
import pygetwindow as gw
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Minimum difference between index finger tip and base Y-coordinates
MIN_RAISE_THRESHOLD = 20  # Adjust this as needed

# Track the last action to prevent duplicates
last_action = None

def activate_presentation_window():
    """Activate the PowerPoint Slide Show window."""
    for window in gw.getAllTitles():
        if "Slide Show" in window:  # Match partial title
            target_window = gw.getWindowsWithTitle(window)[0]
            print(f"Activating window: {target_window.title}")
            target_window.activate()
            return True
    print("Slide Show window not found. Ensure it is open and in full-screen mode.")
    return False

def trigger_action(handedness):
    """ Trigger slide change based on the handedness (left or right hand). """
    global last_action
    if handedness == "Right":
        if activate_presentation_window():
            pyautogui.press('right')
            last_action = "next"
            print("Next Slide (Right Hand Index Finger Raised)")
    elif handedness == "Left":
        if activate_presentation_window():
            pyautogui.press('left')
            last_action = "previous"
            print("Previous Slide (Left Hand Index Finger Raised)")

# Start video capture
cap = cv2.VideoCapture(0)

try:
    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting.")
            break

        # Flip the frame horizontally for a mirrored view
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB (MediaPipe requires RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        results = hands.process(rgb_frame)

        # Check for hand landmarks
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine if the hand is left or right
                handedness = hand_handedness.classification[0].label  # 'Left' or 'Right'

                # Get landmarks for the index finger (landmark 8 is the tip, landmark 5 is the base)
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_base = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]

                h, w, c = frame.shape  # Frame dimensions
                tip_y = int(index_tip.y * h)  # Y-coordinate of index finger tip
                base_y = int(index_base.y * h)  # Y-coordinate of index finger base

                # Determine if the finger is raised enough to trigger an action
                if (base_y - tip_y) > MIN_RAISE_THRESHOLD:
                    # Add visual feedback for detected action
                    color = (0, 255, 0)  # Green color indicates detection
                    position = (50, 50) if handedness == "Right" else (50, 100)
                    label = f"{handedness} Index Finger Detected"

                    # Draw the label on the frame
                    cv2.putText(frame, label, position, cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

                    # Trigger the action if it's not already triggered
                    if handedness == "Right" and last_action != "next":
                        trigger_action(handedness)
                    elif handedness == "Left" and last_action != "previous":
                        trigger_action(handedness)
                else:
                    last_action = None  # Reset action state when the finger is not raised

                # Draw the hand landmarks on the frame
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the video feed with hand landmarks and visual feedback
        cv2.imshow('Hand Detection with Visual Feedback', frame)

        # Press 'q' to quit the application
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger Raised)
Activating window: PowerPoint Slide Show  -  Participant_0

### ROI's on two sides of the Window

Show right index to slide right , left index to slide ppt left.

In [11]:
import cv2
import mediapipe as mp
import pyautogui
import pygetwindow as gw
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Define the screen dimensions and regions of interest (ROIs) for triggering actions
ROI_WIDTH, ROI_HEIGHT = 150, 100  # Dimensions of the bottom right and left regions
roi_bottom_left = [(10, 350), (10 + ROI_WIDTH, 350 + ROI_HEIGHT)]  # Bottom left
roi_bottom_right = [(480, 350), (480 + ROI_WIDTH, 350 + ROI_HEIGHT)]  # Bottom right

# Track the last action to prevent duplicates
last_action = None

# Cooldown parameters
COOLDOWN_TIME = 1.5  # Minimum cooldown time in seconds between actions
last_trigger_time = 0

def activate_presentation_window():
    """Activate the PowerPoint Slide Show window."""
    for window in gw.getAllTitles():
        if "Slide Show" in window:  # Match partial title
            target_window = gw.getWindowsWithTitle(window)[0]
            print(f"Activating window: {target_window.title}")
            target_window.activate()
            return True
    print("Slide Show window not found. Ensure it is open and in full-screen mode.")
    return False

def trigger_action(handedness):
    """Trigger slide change based on the handedness (left or right hand)."""
    global last_action, last_trigger_time
    current_time = time.time()

    # Only allow an action if sufficient time has passed since the last one
    if current_time - last_trigger_time < COOLDOWN_TIME:
        return

    if handedness == "Right":
        if activate_presentation_window():
            pyautogui.press('right')
            last_action = "next"
            last_trigger_time = current_time
            print("Next Slide (Right Hand Index Finger in ROI)")
    elif handedness == "Left":
        if activate_presentation_window():
            pyautogui.press('left')
            last_action = "previous"
            last_trigger_time = current_time
            print("Previous Slide (Left Hand Index Finger in ROI)")

def is_finger_in_roi(x, y, roi):
    """Check if the finger coordinates (x, y) fall within the defined ROI."""
    x1, y1 = roi[0]
    x2, y2 = roi[1]
    return x1 <= x <= x2 and y1 <= y <= y2

# Start video capture
cap = cv2.VideoCapture(0)

try:
    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting.")
            break

        # Flip the frame horizontally for a mirrored view
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB (MediaPipe requires RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        results = hands.process(rgb_frame)

        # Draw ROIs on the frame
        cv2.rectangle(frame, roi_bottom_left[0], roi_bottom_left[1], (255, 0, 0), 2)  # Bottom left ROI (blue)
        cv2.rectangle(frame, roi_bottom_right[0], roi_bottom_right[1], (0, 0, 255), 2)  # Bottom right ROI (red)

        # Check for hand landmarks
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine if the hand is left or right
                handedness = hand_handedness.classification[0].label  # 'Left' or 'Right'

                # Get landmarks for the index finger (landmark 8 is the tip)
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                h, w, c = frame.shape  # Frame dimensions
                tip_x = int(index_tip.x * w)  # X-coordinate of index finger tip
                tip_y = int(index_tip.y * h)  # Y-coordinate of index finger tip

                # Check if the index finger tip is within either ROI
                if is_finger_in_roi(tip_x, tip_y, roi_bottom_left) and handedness == "Left":
                    trigger_action(handedness)
                elif is_finger_in_roi(tip_x, tip_y, roi_bottom_right) and handedness == "Right":
                    trigger_action(handedness)

                # Draw the hand landmarks on the frame
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Visual feedback for detected index finger in ROI
                if is_finger_in_roi(tip_x, tip_y, roi_bottom_left) or is_finger_in_roi(tip_x, tip_y, roi_bottom_right):
                    color = (0, 255, 0)  # Green to indicate detection in ROI
                    cv2.putText(frame, "Finger Detected in ROI", (tip_x, tip_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        # Display the video feed with hand landmarks and visual feedback
        cv2.imshow('Hand Detection with Visual Feedback', frame)

        # Press 'q' to quit the application
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)


### Both ROI's to the right side of the Window

In [12]:
import cv2
import mediapipe as mp
import pyautogui
import pygetwindow as gw
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Define the screen dimensions and regions of interest (ROIs) for triggering actions
frame_width = 1280  # Increased width of the frame for larger view
frame_height = 720  # Increased height of the frame for larger view

# ROI_WIDTH, ROI_HEIGHT = 200, 150  # Dimensions of the right-hand regions
ROI_WIDTH, ROI_HEIGHT = 200, 200  # Dimensions of the right-hand regions

# Position both ROIs side by side on the right-hand side
roi_bottom_left = [(frame_width - 2 * ROI_WIDTH - 20, frame_height - ROI_HEIGHT - 20), (frame_width - ROI_WIDTH - 20, frame_height - 20)]  # Bottom-right left ROI
roi_bottom_right = [(frame_width - ROI_WIDTH - 10, frame_height - ROI_HEIGHT - 20), (frame_width - 10, frame_height - 20)]  # Bottom-right right ROI

# Track the last action to prevent duplicates
last_action = None

# Cooldown parameters
COOLDOWN_TIME = 1.5  # Minimum cooldown time in seconds between actions
last_trigger_time = 0

def activate_presentation_window():
    """Activate the PowerPoint Slide Show window."""
    for window in gw.getAllTitles():
        if "Slide Show" in window:  # Match partial title
            target_window = gw.getWindowsWithTitle(window)[0]
            print(f"Activating window: {target_window.title}")
            target_window.activate()
            return True
    print("Slide Show window not found. Ensure it is open and in full-screen mode.")
    return False

def trigger_action(handedness):
    """Trigger slide change based on the handedness (left or right hand)."""
    global last_action, last_trigger_time
    current_time = time.time()

    # Only allow an action if sufficient time has passed since the last one
    if current_time - last_trigger_time < COOLDOWN_TIME:
        return

    if handedness == "Right":
        if activate_presentation_window():
            pyautogui.press('right')
            last_action = "next"
            last_trigger_time = current_time
            print("Next Slide (Right Hand Index Finger in ROI)")
    elif handedness == "Left":
        if activate_presentation_window():
            pyautogui.press('left')
            last_action = "previous"
            last_trigger_time = current_time
            print("Previous Slide (Left Hand Index Finger in ROI)")

def is_finger_in_roi(x, y, roi):
    """Check if the finger coordinates (x, y) fall within the defined ROI."""
    x1, y1 = roi[0]
    x2, y2 = roi[1]
    return x1 <= x <= x2 and y1 <= y <= y2

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

try:
    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting.")
            break

        # Flip the frame horizontally for a mirrored view
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB (MediaPipe requires RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        results = hands.process(rgb_frame)

        # Draw ROIs on the frame
        cv2.rectangle(frame, roi_bottom_left[0], roi_bottom_left[1], (255, 0, 0), 2)  # Bottom right left ROI (blue)
        cv2.rectangle(frame, roi_bottom_right[0], roi_bottom_right[1], (0, 0, 255), 2)  # Bottom right right ROI (red)

        # Check for hand landmarks
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine if the hand is left or right
                handedness = hand_handedness.classification[0].label  # 'Left' or 'Right'

                # Get landmarks for the index finger (landmark 8 is the tip)
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                h, w, c = frame.shape  # Frame dimensions
                tip_x = int(index_tip.x * w)  # X-coordinate of index finger tip
                tip_y = int(index_tip.y * h)  # Y-coordinate of index finger tip

                # Check if the index finger tip is within either ROI
                if is_finger_in_roi(tip_x, tip_y, roi_bottom_left) and handedness == "Left":
                    trigger_action(handedness)
                elif is_finger_in_roi(tip_x, tip_y, roi_bottom_right) and handedness == "Right":
                    trigger_action(handedness)

                # Draw the hand landmarks on the frame
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Visual feedback for detected index finger in ROI
                if is_finger_in_roi(tip_x, tip_y, roi_bottom_left) or is_finger_in_roi(tip_x, tip_y, roi_bottom_right):
                    color = (0, 255, 0)  # Green to indicate detection in ROI
                    cv2.putText(frame, "Finger Detected in ROI", (tip_x, tip_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        # Display the video feed with hand landmarks and visual feedback
        cv2.imshow('Hand Detection with Visual Feedback', frame)

        # Press 'q' to quit the application
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Left Hand Index Finger in ROI)
Activating window: PowerPoint Slide Show  -  Part

### Right ROI moves right; Left ROI moves left showing the Right index finger only.

In [14]:
import cv2
import mediapipe as mp
import pyautogui
import pygetwindow as gw
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Define the screen dimensions and regions of interest (ROIs) for triggering actions
frame_width = 1280  # Increased width of the frame for larger view
frame_height = 720  # Increased height of the frame for larger view

# ROI_WIDTH, ROI_HEIGHT = 200, 150  # Dimensions of the right-hand regions
ROI_WIDTH, ROI_HEIGHT = 200, 200  # Dimensions of the right-hand regions

# Position both ROIs side by side on the right-hand side
roi_bottom_left = [(frame_width - 2 * ROI_WIDTH - 20, frame_height - ROI_HEIGHT - 20), 
                   (frame_width - ROI_WIDTH - 20, frame_height - 20)]  # Bottom-right left ROI
roi_bottom_right = [(frame_width - ROI_WIDTH - 10, frame_height - ROI_HEIGHT - 20), 
                    (frame_width - 10, frame_height - 20)]  # Bottom-right right ROI

# Track the last action to prevent duplicates
last_action = None

# Cooldown parameters
COOLDOWN_TIME = 1.5  # Minimum cooldown time in seconds between actions
last_trigger_time = 0

def activate_presentation_window():
    """Activate the PowerPoint Slide Show window."""
    for window in gw.getAllTitles():
        if "Slide Show" in window:  # Match partial title
            target_window = gw.getWindowsWithTitle(window)[0]
            print(f"Activating window: {target_window.title}")
            target_window.activate()
            return True
    print("Slide Show window not found. Ensure it is open and in full-screen mode.")
    return False

def trigger_action(action):
    """Trigger slide change action based on the given action ('next' or 'previous')."""
    global last_action, last_trigger_time
    current_time = time.time()

    # Only allow an action if sufficient time has passed since the last one
    if current_time - last_trigger_time < COOLDOWN_TIME:
        return

    if action == "next":
        if activate_presentation_window():
            pyautogui.press('right')
            last_action = "next"
            last_trigger_time = current_time
            print("Next Slide (Right Hand Index Finger in Right ROI)")
    elif action == "previous":
        if activate_presentation_window():
            pyautogui.press('left')
            last_action = "previous"
            last_trigger_time = current_time
            print("Previous Slide (Right Hand Index Finger in Left ROI)")

def is_finger_in_roi(x, y, roi):
    """Check if the finger coordinates (x, y) fall within the defined ROI."""
    x1, y1 = roi[0]
    x2, y2 = roi[1]
    return x1 <= x <= x2 and y1 <= y <= y2

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

try:
    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting.")
            break

        # Flip the frame horizontally for a mirrored view
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB (MediaPipe requires RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        results = hands.process(rgb_frame)

        # Draw ROIs on the frame
        cv2.rectangle(frame, roi_bottom_left[0], roi_bottom_left[1], (255, 0, 0), 2)  # Bottom-right left ROI (blue)
        cv2.rectangle(frame, roi_bottom_right[0], roi_bottom_right[1], (0, 0, 255), 2)  # Bottom-right right ROI (red)

        # Check for hand landmarks
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine if the hand is left or right
                handedness = hand_handedness.classification[0].label  # 'Left' or 'Right'

                # Get landmarks for the index finger (landmark 8 is the tip)
                if handedness == "Right":
                    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                    h, w, c = frame.shape  # Frame dimensions
                    tip_x = int(index_tip.x * w)  # X-coordinate of index finger tip
                    tip_y = int(index_tip.y * h)  # Y-coordinate of index finger tip

                    # Check if the index finger tip is within either ROI
                    if is_finger_in_roi(tip_x, tip_y, roi_bottom_left):
                        trigger_action("previous")
                        cv2.putText(frame, "Previous Slide Triggered", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    elif is_finger_in_roi(tip_x, tip_y, roi_bottom_right):
                        trigger_action("next")
                        cv2.putText(frame, "Next Slide Triggered", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                    # Visual feedback for detected index finger in ROI
                    if is_finger_in_roi(tip_x, tip_y, roi_bottom_left) or is_finger_in_roi(tip_x, tip_y, roi_bottom_right):
                        color = (0, 255, 0)  # Green to indicate detection in ROI
                        cv2.putText(frame, "Finger Detected in ROI", (tip_x, tip_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

                # Draw the hand landmarks on the frame
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the video feed with hand landmarks and visual feedback
        cv2.imshow('Hand Detection with Visual Feedback', frame)

        # Press 'q' to quit the application
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in Right ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in Right ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Next Slide (Right Hand Index Finger in Right ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Right Hand Index Finger in Left ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Right Hand Index Finger in Left ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Right Hand Index Finger in Left ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Right Hand Index Finger in Left ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Right Hand Index Finger in Left ROI)
Activating window: PowerPoint Slide Show  -  Participant_01
Previous Slide (Right Hand Index Finger in Le